| Что удаляем           | Код                                   |
| --------------------- | ------------------------------------- |
| URL                   | re.sub(r'https?://\S+', '', text)     |
| Email                 | re.sub(r'\S+@\S+', '', text)          |
| HTML-теги             | re.sub(r'<.\*?>', '', text)           |
| Эмодзи                | re.sub(r'\[^\w\s,!?;:.()]', '', text) |
| Повторяющиеся пробелы | re.sub(r'\s+', ' ', text).strip()     |
| Ненужные символы      | re.sub(r'\[^\x00-\x7F]+', '', text)   |


| Что делать                | Пример                          |
| ------------------------- | ------------------------------- |
| .lower()                  | 'Hello' → 'hello'               |
| strip()                   | убирает пробелы по краям        |
| удаление пунктуации       | re.sub(r'\[^\w\s]', '', text)   |
| замена цифр на спец.токен | re.sub(r'\d+', ' <NUM> ', text) |


In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
text = 'karim parim'
words = text.split()
filtered = ' '.join([w for w in words if w.lower() not in ENGLISH_STOP_WORDS])


In [ ]:
import re

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+', '', text)  # remove urls
    text = re.sub(r'<.*?>', '', text)         # remove html
    text = re.sub(r'[^a-z0-9\s]', '', text)   # remove punct
    text = re.sub(r'\d+', ' <NUM> ', text)    # numbers → token
    text = re.sub(r'\s+', ' ', text).strip()  # spaces
    return text


In [ ]:
import torchvision.transforms as T

train_transforms = T.Compose([
    T.Resize((256, 256)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(brightness=0.2, contrast=0.2),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])


train2_transforms = T.Compose([
    T.Resize((256, 256)),  # или RandomResizedCrop для увеличения разнообразия
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    T.RandomPerspective(distortion_scale=0.2, p=0.5),  # имитация 3D
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])



| Трюк                            | Что делает                                 |
| ------------------------------- | ------------------------------------------ |
| Label encoding                  | превращает категорию в число               |
| One-hot encoding                | если мало уникальных значений              |
| Frequency / Count encoding      | полезно для LightGBM / CatBoost            |
| Mean encoding (target encoding) | заменяет категорию на среднее таргета      |
| Interaction features            | `feature1 * feature2`, `f1/f2` и т.д.      |
| Date features                   | `month`, `dayofweek`, `is_weekend`, `hour` |


| Что делать                           | Почему важно                                  |
| ------------------------------------ | --------------------------------------------- |
| Замена на медиану / среднее          | особенно для чисел                            |
| Заполнение “None” или “missing”      | для категориальных                            |
| Отдельный бинарный признак `is_null` | помогает моделям понимать, что было пропущено |


| Метод     | Как                                                 |
| --------- | --------------------------------------------------- |
| z-score   | `abs((x - mean)/std) > 3` → потенциально выброс     |
| IQR       | `x < Q1 - 1.5*IQR` или `x > Q3 + 1.5*IQR`           |
| Визуально | boxplot / scatter plot → удаляй вручную, если нужно |


📌 Regression (регрессия)
"regression" или "regression_l2" — MSE (L2 loss)

"regression_l1" — MAE (L1 loss)

"huber" — Huber loss

"fair" — Fair loss

"poisson" — Poisson regression (для счетных данных)

"quantile" — Quantile regression (например, для предсказания медианы)

"mape" — Mean Absolute Percentage Error

"gamma" — Gamma regression

"tweedie" — Tweedie distribution regression

📌 Binary Classification
"binary" — логистическая регрессия (бинарная классификация)

📌 Multiclass Classification
"multiclass" — мультиклассовая классификация

"multiclassova" — One-vs-All (OvA) стратегия для мультикласса

📌 Ranking
"lambdarank" — LambdaRank (с NDCG)

"rank_xendcg" — XENDCG (более стабильный аналог LambdaRank)

"rank_gbdt" — GBDT Rank (обычный boosting для ранжирования)

📌 Custom
"none" — без встроенной функции ошибки (нужно передать feval)



In [ ]:
import numpy as np

def remove_outliers_zscore(df, column, threshold=3.0):
    mean = df[column].mean()
    std = df[column].std()
    z_scores = (df[column] - mean) / std
    return df[np.abs(z_scores) < threshold]


In [ ]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]
